# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_csv = "../output_data/cities.csv"
clean_city_data_df = pd.read_csv(cities_csv)
clean_city_data_df = clean_city_data_df.set_index("City_ID")
clean_city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,Tasiilaq,65.6145,-37.6368,26.91,67,0,8.01,GL,1604350800
1,Am Timan,11.0297,20.2827,81.81,34,29,2.55,TD,1604350800
2,Evensk,61.9500,159.2333,25.47,97,99,5.46,RU,1604350800
3,Wollongong,-34.4333,150.8833,61.52,80,24,1.79,AU,1604350800
4,Punta Arenas,-53.1500,-70.9167,56.21,56,87,2.42,CL,1604350800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
latlng_df = clean_city_data_df[["Lat", "Lng"]].astype(float)
latlng_df.head()
humidity_df = clean_city_data_df["Humidity"].astype(float)
humidity_df.head()

City_ID
0    67.0
1    34.0
2    97.0
3    80.0
4    56.0
Name: Humidity, dtype: float64

In [11]:
fig = gmaps.figure(center=(50.0, 0.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(latlng_df, weights=humidity_df, max_intensity=100, 
                                    point_radius=3.0, dissipating=False, opacity=.5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 80 degrees but higher than 70
    * Wind speed less than 10 mph.
    * Zero cloudiness.
* Drop any rows will null values.

In [ ]:
hotel_df = clean_city_data_df.loc[((clean_city_data_df["Max Temp"] < 80 & clean_city_data_df["Max Temp"] > 70) &
                                   (clean_city_data_df["Wind Speed"] < 10) &
                                   (clean_city_data_df["Cloudiness"] == 0)), :]
hotel_df = hotel_df.dropna(how="any")
hotel_df.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?location="

In [ ]:
test_url = f"{base_url}65.6145,-37.6368&radius=5000&type=hotel&key={g_key}"
response = requests.get(test_url)
data = response.json()
print(json.dumps(data, indent=4, sort_keys=True))

# Iterate over the rows in the dataframe
# for index, row in hotel_df.iterrows():
# 
#     # get restaurant type from df
#     city = row['City']
#     country = row['Country']
#     lat = row['Lat']
#     lng = row['Lng']
#     
#     # set up the url
#     url = f"{base_url}{lat},{lng}&radius=5000&type=hotel&key={g_key}"
#     
#     # Make API request
#     print(f"Retrieving Results for Index {index}: {city}.")
#     response = requests.get(url)
#     data = response.json()
#     
#     # Get the results
#     results = data['results']
#     
#     try:
#         name = response['results'][7]['name']
#         print(f"Nearest hotel to {city} at is {name}.")
#         
#         hotel_df.loc[index, "Hotel Name"] = name
#         
#     except (KeyError, IndexError):
#         print(f"Error with city={city}, skipping...")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
